We compare minimum average particle distance acorss trajecotry, last-step average particle distance across trajecotries, minimum average semantic distance across trajecotries, last-step average particle distance across trajectories, max_IoU agains the goal, automatically detected SR. We fix the trajecotry horizon as 20 regardless if it reaches success or not. Because, we also want to test the policies capability not to mess the success states. The evalaution arena should not have readjust pick assistance.

TODO:
1. Regenerate the oracle folding with fixed 20 horizon.
2. Generate the huma policy
3. in hydra train and eval, seperate the train trajectory setup from the eval trajecotry setup, because some algo needs the reajust-pick assistance.
    * use ray-capsulated arena for pyflex.

In [ ]:
import os
import pandas as pd
import numpy as np
import ast


# -------------------------------
# HELPER: parse list-like strings
# -------------------------------

def parse_list(s):
    """Convert string like '[0.2, 0.1, 0.05]' into list of floats."""
    if isinstance(s, list):
        return s
    return ast.literal_eval(s)


# -------------------------------
# METRIC EXTRACTION LOGIC
# -------------------------------
def aggregate_performance(csv_path):
    df = pd.read_csv(csv_path)

    # Convert list-like string columns to actual lists
    df["particle_dist"] = df["evaluation/mean_particle_distance"].apply(parse_list)
    df["semantic_dist"] = df["evaluation/semantic_keypoint_distance"].apply(parse_list)
    df["iou"] = df["evaluation/max_IoU"].apply(parse_list)
    df["success_list"] = df["evaluation/success"].apply(parse_list)

    # ---------------------------
    # Extract per-row statistics
    # ---------------------------

    # 1. Min across each step's list
    min_particle_distance = [np.min(x) for x in df["particle_dist"]]
    min_semantic_distance = [np.min(x) for x in df["semantic_dist"]]
    max_max_iou = [np.max(x) for x in df["iou"]]         # max IoU per row

    # 2. Last-step metrics
    last_particle_distance = [x[-1] for x in df["particle_dist"]]
    last_semantic_distance = [x[-1] for x in df["semantic_dist"]]
    last_max_iou = [x[-1] for x in df["iou"]]

    # 3. Success for each trajectory
    success_flags = [int(any(x)) for x in df["success_list"]]

    # ---------------------------
    # Aggregated metrics (means)
    # ---------------------------
    min_particle_mean = np.mean(min_particle_distance)
    last_particle_mean = np.mean(last_particle_distance)

    min_semantic_mean = np.mean(min_semantic_distance)
    last_semantic_mean = np.mean(last_semantic_distance)

    max_iou_mean = np.mean(max_max_iou)
    last_iou_mean = np.mean(last_max_iou)

    success_rate = np.mean(success_flags)

    # ---------------------------
    # Standard deviations
    # ---------------------------
    min_particle_std = np.std(min_particle_distance)
    last_particle_std = np.std(last_particle_distance)

    min_semantic_std = np.std(min_semantic_distance)
    last_semantic_std = np.std(last_semantic_distance)

    max_iou_std = np.std(max_max_iou)
    last_iou_std = np.std(last_max_iou)

    success_rate_std = np.std(success_flags)

    return {
        # Means
        "min_particle_distance_mean": min_particle_mean,
        "last_particle_distance_mean": last_particle_mean,
        "min_semantic_distance_mean": min_semantic_mean,
        "last_semantic_distance_mean": last_semantic_mean,
        "max_IoU_mean": max_iou_mean,
        "last_IoU_mean": last_iou_mean,
        "success_rate": success_rate,

        # Standard deviations
        "min_particle_distance_std": min_particle_std,
        "last_particle_distance_std": last_particle_std,
        "min_semantic_distance_std": min_semantic_std,
        "last_semantic_distance_std": last_semantic_std,
        "max_IoU_std": max_iou_std,
        "last_IoU_std": last_iou_std,
        "success_rate_std": success_rate_std,
    }


In [2]:

data_dir = "/mnt/ssd/garment_folding_data"
experiments = [
    {
        "name": "oracle-centre-sleeve-folding-from-flattened",
        "label": "Oracle Centre Sleeve Folding"
    }
]

rows = []

for exp in experiments:
    csv_path = os.path.join(
        data_dir,
        exp["name"],
        "eval_checkpoint_-1",
        "performance.csv"
    )
    print('csv', csv_path)
    if not os.path.exists(csv_path):
        print(f"WARNING: Missing file: {csv_path}")
        continue

    metrics = aggregate_performance(csv_path)
    metrics["experiment"] = exp["label"]
    rows.append(metrics)

results_df = pd.DataFrame(rows)

display(results_df)

csv /mnt/ssd/garment_folding_data/oracle-centre-sleeve-folding-from-flattened/eval_checkpoint_-1/performance.csv


,min_particle_distance_mean,last_particle_distance_mean,min_semantic_distance_mean,last_semantic_distance_mean,max_IoU_mean,last_IoU_mean,success_rate,min_particle_distance_std,last_particle_distance_std,min_semantic_distance_std,last_semantic_distance_std,max_IoU_std,last_IoU_std,success_rate_std,experiment
0,0.072924,0.072924,0.087813,0.087813,0.784022,0.775784,0.461538,0.039522,0.039522,0.038922,0.038922,0.087081,0.095084,0.498519,Oracle Centre Sleeve Folding
